# Welcome to HTMX cookbook

> Please note, this very much WIP. If you have certain recipes that you want to share or want to see, head to [Github](https://github.com/callmephilip/htmx-cookbook)

This cookbook is powered by [FastHTML](https://fastht.ml/). UI is based on [block.css](https://github.com/thesephist/blocks.css)

## Getting started

```
python -m venv .env
source .env/bin/activate
pip install notebook
jupyter notebook
```

The easiest way to run this is to smash `Run -> Run All Cells`. To iterate on a specific recipe, update code in the corresponding cell and rerun it

![Screenshot](./cookbook.gif)

In [1]:
! pip install --no-cache-dir git+https://github.com/callmephilip/fasthtml.git@tweak-jupyter-integration password_strength

  Cloning https://github.com/callmephilip/fasthtml.git (to revision tweak-jupyter-integration) to /private/var/folders/6_/kmpf38495hzcv9549vccky700000gn/T/pip-req-build-zd7pf0jf
  Running command git clone --filter=blob:none --quiet https://github.com/callmephilip/fasthtml.git /private/var/folders/6_/kmpf38495hzcv9549vccky700000gn/T/pip-req-build-zd7pf0jf
  Running command git checkout -b tweak-jupyter-integration --track origin/tweak-jupyter-integration
  Switched to a new branch 'tweak-jupyter-integration'
  branch 'tweak-jupyter-integration' set up to track 'origin/tweak-jupyter-integration'.
  Resolved https://github.com/callmephilip/fasthtml.git to commit 3bf075a96fe8ffc2c4928d61e78f10689100a757
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# Setup - you can largely ignore this unless you wan to tweak how the cookbook works

import os, time, re, traceback
from urllib.parse import urlparse
from urllib.request import urlopen
from typing import Optional
from fasthtml.common import *
from fasthtml.jupyter import *

# UI
def prettify(f):
    def _f(*args, **kw): return f(*args, **(kw | { "cls": "block " + kw.get("cls","") }))
    return _f

@prettify
def Panel(*content, **kw): return Div(*content, **kw)

@prettify
def Textbox(*content, **kw): return Input(*content, **kw)

@prettify
def Password(*content, **kw): return Input(type="password", *content, **kw)

@prettify
def MultilineTextbox(*content, **kw): return Textarea(*content, **kw)

@prettify
def ActionButton(*content, **kw): return Button(*content, **kw)

def AddressBar(*content, **kw): return Textbox(*content, readonly=True, **kw)

on_response_error = """document.querySelector("#errors").innerHTML = event.detail.xhr.responseText; document.querySelector(".error-box").classList.add("show");"""

def Playground(*content, path:str): return ErrorBox(), Panel(AddressBar(value=f"🌎 http://localhost:8000{path}", cls="w-100"), Hr(), *content, HtmxOn("responseError", on_response_error), cls="h-fullish")
def ErrorBox(): return Panel(Div(Div(P("Oh 🦌!")), Div(ActionButton("x", cls="display-inline", onclick="document.querySelector('.error-box').classList.remove('show')"), cls="flex-grow-1 text-align-right"), cls="flex flex-row"), Hr(), P(id="errors"), cls="error-box")


class ColabFriendlyJupyUvi(JupyUvi):
  def start(self, url="http://localhost:8000",max_attempts=5,initial_delay=1,out="tunnel.log") -> Optional[str]:
    super(ColabFriendlyJupyUvi, self).start()
    try:
      import google.colab

      # setup cloudflare tunnel: https://pkg.cloudflare.com/index.html
      installation_script = """
      sudo mkdir -p --mode=0755 /usr/share/keyrings
      curl -fsSL https://pkg.cloudflare.com/cloudflare-main.gpg | sudo tee /usr/share/keyrings/cloudflare-main.gpg >/dev/null
      echo 'deb [signed-by=/usr/share/keyrings/cloudflare-main.gpg] https://pkg.cloudflare.com/cloudflared focal main' | sudo tee /etc/apt/sources.list.d/cloudflared.list
      sudo apt-get update && sudo apt-get install cloudflared
      """
      if os.system("cloudflared --version") != 0: os.system(installation_script)
      os.system(f"nohup cloudflared tunnel --url {url} > {out} 2>&1 &")
      time.sleep(5)
      attempt = 0
      delay = initial_delay

      while attempt < max_attempts:
          try:
              with open(out) as f:
                for l in f.read().split("\n"):
                  log_entry = l
                  url_pattern = r"https?://[^\s]*trycloudflare\.com[^\s]*"
                  url_match = re.search(url_pattern, log_entry)
                  if url_match: return urlparse(url_match.group(0)).hostname
                raise ValueError("URL not found")
          except:
              attempt += 1
              if attempt < max_attempts:
                  time.sleep(delay)
                  delay *= 2  # Exponential backoff
              else: return None
    except: return None


# scripts
htmx,fasthtml_js = "https://unpkg.com/htmx.org@2.0.2", "https://cdn.jsdelivr.net/gh/answerdotai/fasthtml-js@1.0.4/fasthtml.js"
# styles + fonts
styles,blk_css,font_css = "/styles.css","https://unpkg.com/blocks.css/dist/blocks.min.css","https://fonts.googleapis.com/css2?family=IBM+Plex+Mono:wght@500;700&amp;display=swap"
inline_styles = [Style(urlopen("https://raw.githubusercontent.com/callmephilip/htmx-cookbook/refs/heads/main/styles.css").read().decode('utf-8'))]
hdrs = [Script(src=s) for s in [htmx, fasthtml_js]] + [Link(href=style, rel="stylesheet") for style in [blk_css,styles,font_css]] + inline_styles

def server_error(request: Request, exc: HTTPException): return HTMLResponse(content=''.join(traceback.format_exception(exc)), status_code=500)

app = FastJupy(default_hdrs=False, hdrs=hdrs, exception_handlers={ 500: server_error, Exception: server_error })
rt, server = app.route, ColabFriendlyJupyUvi(app, port=8000, start=False)
app_host = server.start()

def run_cookbook_recipe(path: str): return HTMX(host=app_host or "localhost", path=path, protocol="https" if app_host else "http")

In [3]:
@rt(path="/")
def get():
    return Playground(
        H1("HTMX cookbook"),
        P("This is very much WIP. Please leave your comments/requests on Github"),
        Div("Let's get started 👇", cls="wrapper animate-bounce block"),
        path="/"
    )

run_cookbook_recipe(path="/")

# Basic action

Grab some stuff from a URL on click

In [4]:
@rt("/data")
def get(): return Span("😱 i am trapped inside the button now!")

@rt(path="/get-on-click")
def get(): return Playground(ActionButton("Get some data", hx_get="/data"), path="/get-on-click")

run_cookbook_recipe(path="/get-on-click")

# Custom trigger attribute 

In [5]:
from time import sleep

@dataclass
class Html2FT: html: str

@rt("/html2ft/convert")
def post(data: Html2FT): 
    sleep(5)
    return html2ft(data.html) 

on_paste = """
document.querySelector("#txt-html").addEventListener("paste", function (event) { setTimeout(() => {event.target.blur();}, 500) });
"""

@rt("/html2ft")
def get():
    return Playground(
        Div(
            Div(
                P("Convert HTML to FT"),
                MultilineTextbox(name="html", placeholder="paste html here", cls="w-100 h-80", id="txt-html", hx_post="/html2ft/convert", hx_trigger="blur", hx_target="#result", hx_indicator="#processing-indicator"),
                Strong("Processing...", id="processing-indicator", cls="htmx-indicator"),
                cls="flex-grow-1 h-300px"
            ),
            Div(
                Code(id="result", cls="white-space-pre-wrap text-align-left"),
                cls="flex-grow-2 h-300px text-align-left", style="padding-left: 5rem;"
            ),
            cls="flex flex-row"
        ),
        Script(code=on_paste),
        path="/html2ft"
    )

run_cookbook_recipe(path="/html2ft")

# You can also post some data over

In [6]:
from password_strength import PasswordStats

@dataclass
class LoginData: email: str; pw:str 

@rt("/login")
def post(ld: LoginData): return Span(f"You are logged in!")

@rt("/login/validation")
def post(ld: LoginData): 
    strength = int(PasswordStats(ld.pw).strength() * 10) if ld.pw else 0
    if strength > 7:
        c = "green"
    elif strength > 5:
        c = "orange"
    else: c = "red"
    
    return Div("✅" if "@" in ld.email else "🔴", id="email-validator", hx_swap_oob="true"), strength > 0 and Div(
        Div(*[Div("+", cls=f"text-{c}") if i + 1 <= strength else Div("_", cls="tex-gray") for i in range(9)], cls="flex flex-row"),
        id="password-validator",
        hx_swap_oob="true"
    )

@rt("/login")
def get():
    return Playground(
        Form(
            Div(
                Textbox(name="email", placeholder="Email", hx_post="/login/validation", hx_trigger="keyup"), 
                Div(id="email-validator"),
                cls="flex flex-row"
            ),
            Div(
                Password(name="pw", placeholder="Password", hx_post="/login/validation", hx_trigger="keyup"),
                Div(id="password-validator"),
                cls="flex flex-row"
            ),
            ActionButton("Go!"),
            hx_post="/login"
        ), 
        path="/login")
 
run_cookbook_recipe(path="/login")